In [1]:
%time 

import os
import sys

PROJECT_PATH = '/Users/orprager/PycharmProjects/ml'
module_path = os.path.abspath('/Users/orprager/PycharmProjects/ml')
if module_path not in sys.path:
    print "adding {} to sys.path".format(module_path)
    sys.path.insert(1, module_path)


CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs


In [3]:
%time

# Loading the reviews
import pandas as pd
from samples.movies_reviews_sentiment_analysis import get_labeled_train_data, get_test_unlabeled_data, \
    get_unlabeled_train_data

labeled_train_review = get_labeled_train_data()
test_reviews = get_test_unlabeled_data()
unlabeled_train_reviews = get_unlabeled_train_data()

print "labeled reviews: {}".format(labeled_train_review.shape)
print "test reviews: {}".format(test_reviews.shape)
print "unlabeled reviews: {}".format(unlabeled_train_reviews.shape)

print "\nlabeled reviews columns: {}".format(labeled_train_review.columns.values)


CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 21 µs


labeled reviews: (25000, 3)
test reviews: (25000, 2)
unlabeled reviews: (50000, 2)

labeled reviews columns: ['id' 'sentiment' 'review']


In [4]:
%time
# Review to list of sentences (word2vec expect to get the sentences separated)

from lib.nlp import text_to_sentences
import nltk

# NLTK has a tokenizer allow us to split text by sentences
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

sentences = []

print "Parsing sentences from labeled training set"
for review in labeled_train_review["review"]:
    sentences += text_to_sentences(review, sentence_tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train_reviews["review"]:
    sentences += text_to_sentences(review, sentence_tokenizer)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


Parsing sentences from labeled training set


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


...

/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.youtube.com/watch?v=a0KSqelmgN8"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [5]:
%time

# Word2vec training
from gensim.models import Word2Vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300  # Word vector dimensionality                      
min_word_count = 40  # min word count to include in vocabulary                        
num_workers = 4  # Number of threads to run in parallel
context = 10  # Context window size - the max size between predicated word to other in same context                                                                                   
downsampling = 1e-3  # Downsample setting for frequent words

model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count,
                 window=context, sample=downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "{0}features_{1}minwords_{2}context".format(num_features, min_word_count, context)
model.wv.save_word2vec_format(model_name)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


2018-03-04 10:58:57,073 : INFO : collecting all words and their counts


2018-03-04 10:58:57,075 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2018-03-04 10:58:57,164 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


2018-03-04 10:58:57,290 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


...

2018-03-04 11:00:30,020 : INFO : worker thread finished; awaiting finish of 0 more threads


2018-03-04 11:00:30,022 : INFO : EPOCH - 5 : training on 17798263 raw words (12750020 effective words) took 19.0s, 672528 effective words/s


2018-03-04 11:00:30,024 : INFO : training on a 88991315 raw words (63754570 effective words) took 87.3s, 730339 effective words/s


2018-03-04 11:00:30,025 : INFO : precomputing L2-norms of word weight vectors


2018-03-04 11:00:30,175 : INFO : storing 16490x300 projection weights into 300features_40minwords_10context


In [6]:
%time
# Here we can play with our model

model.most_similar("good", topn=10)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[(u'decent', 0.6794548630714417),
 (u'bad', 0.6264511942863464),
 (u'great', 0.6156489849090576),
 (u'nice', 0.5974811315536499),
 (u'lousy', 0.5720666646957397),
 (u'fine', 0.5697389841079712),
 (u'passable', 0.562705934047699),
 (u'cool', 0.5605120658874512),
 (u'mediocre', 0.549391508102417),
 (u'terrible', 0.5140118598937988)]

In [7]:
%time
# e.g: we can pull out of our model the word happy and see it's vector representation 
# model['happy'].shape
import os
from gensim.models import KeyedVectors

model_name = os.path.join(PROJECT_PATH, "samples/movies_reviews_sentiment_analysis", "300features_40minwords_10context")
model = KeyedVectors.load_word2vec_format(model_name, binary=True)

2018-03-04 11:00:54,432 : INFO : loading projection weights from /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context


2018-03-04 11:00:54,436 : WARNING : duplicate word '' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


2018-03-04 11:00:54,438 : WARNING : duplicate word '' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


2018-03-04 11:00:54,440 : WARNING : duplicate word '' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


2018-03-04 11:00:54,441 : WARNING : duplicate word '' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


...

2018-03-04 11:00:58,179 : WARNING : duplicate word '43' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


2018-03-04 11:00:58,180 : WARNING : duplicate word '05' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


2018-03-04 11:00:58,184 : WARNING : duplicate word '328' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


2018-03-04 11:00:58,188 : WARNING : duplicate word '' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


2018-03-04 11:00:58,190 : WARNING : duplicate word '348' in /Users/orprager/PycharmProjects/ml/samples/movies_reviews_sentiment_analysis/300features_40minwords_10context, ignoring all but first


In [11]:
%time

import numpy as np  # Make sure that numpy is imported

from lib.nlp import words_vec_to_avg_feature_vec

def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        #
        # Print a status message every 1000th review
        if counter % 1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))
        # 
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = words_vec_to_avg_feature_vec(review, model, \
                                                    num_features)
        #
        # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs


ImportError: cannot import name words_vec_to_avg_feature_vec

In [10]:
%time
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in labeled_train_review["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test_reviews["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

NameError: name 'num_features' is not defined

In [ ]:
%time
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)

print "Fitting a random forest to labeled training data..."
forest = forest.fit(trainDataVecs, labeled_train_review["sentiment"])

# Test & extract results 
result = forest.predict(testDataVecs)

# Write the test results 
output = pd.DataFrame(data={"id": test_reviews["id"], "sentiment": result})
output.to_csv("./data/movie_reviews/Word2Vec_AverageVectors.csv", index=False, quoting=3)
